In [2]:
import pandas as pd
import numpy as np
from linearmodels.iv import IV2SLS
import statsmodels.api as sm

Notes:
1. The fact there's negative utility just means on average the utility is less than if the person did not buy a car. The unobserved characteristics are in the expression for the mean utility. 
2. Oh I actually meant not the unobserved characteristics but the error term must be a positive shock for the person to buy a car like in the marriage model 
3. Income should not be included as it's an individual characteristic that cancels out. Mistake to include. However, you can include an interaction of income with a characteristic 
4. Sales is not a characteristic

In [3]:
# Path to your file
file_path = '/Users/danielseymour/Downloads/car_data_up.dta'

# Load the .dta (Stata) file
df = pd.read_stata(file_path)

# Display the first few rows
df.head()

,brand,model,sales,home,fuel,time,products,age,enginesize,income,averagesizecompetitor,price
0,BMW,5,160,0,0,1994,43,2,3,4045.563721,1870.260010,13251.009766
1,BMW,3,210,1,1,1994,8,2,3,4045.563721,1782.649658,8996.839844
2,daihatsu,applause,370,0,0,1994,34,4,2,4045.563721,1470.613037,5924.431152
3,daihatsu,domino,120,0,1,1994,19,10,3,4045.563721,1907.504395,3472.578857
4,daihatsu,charade,310,0,1,1994,22,5,2,4045.563721,1512.156616,4081.043701


In [4]:
df.describe()

,sales,home,fuel,time,products,age,enginesize,income,averagesizecompetitor,price
count,1206.000000,1206.000000,1206.000000,1206.000000,1206.000000,1206.000000,1206.000000,1206.000000,1206.000000,1206.000000
mean,500.588723,0.495854,0.398010,1998.990050,22.854892,5.009950,2.195688,4292.482422,1436.866455,6539.218750
std,880.956898,0.500190,0.489691,3.163836,21.388515,3.163836,0.754416,665.153198,321.507111,4788.465820
min,20.000000,0.000000,0.000000,1994.000000,0.000000,0.000000,1.000000,3002.935547,286.301941,400.000000
25%,130.000000,0.000000,0.000000,1996.000000,8.000000,2.000000,2.000000,3996.340332,1224.235687,2986.719055
50%,240.000000,0.000000,0.000000,1999.000000,15.000000,5.000000,2.000000,4157.279785,1456.350159,5210.430420
75%,460.000000,1.000000,1.000000,2002.000000,32.000000,8.000000,3.000000,4956.622070,1686.146179,8724.577393
max,8750.000000,1.000000,1.000000,2004.000000,139.000000,10.000000,3.000000,5637.940430,2179.595947,27217.203125


## Step 1: Compute the market shares

We do this based off of unit sales, not revenue. This is because we're modelling the probability that a consumer chooses a product. Using revenue would bias towards more expensive products. We calculate the market shares as:


$$
s_{jt} = \frac{\text{sales}_{jt}}{\text{market size}_t}
$$

Importantly we need to estimate the market size. It's not the sum of the car sales as we need to include the outside good, which is that the consumer buys no car in the dataset as they don't need one, keep their old one, take the bus etc.   

As we're given the data for the total population (79,000) and Adult population (70,000), we can use the adult population to give the potential market size. 

So for each year t:

$$\text{Market share of product } j = \frac{\text{sales}_j}{70,000}$$

$$\text{Market share of outside good } = 1 - \sum_j \left( \frac{\text{sales}_j}{70,000} \right)$$

Note on why we subscript by time:  
- Each year is a separate market in a discrete choice framework.
- Consumers are choosing among cars available in that year.
- The MNL model assumes the choice set and conditions are fixed within each “market” — so you treat each year as a different market (with its own prices, products, sales, etc.).

In [5]:
total_sales_per_year = df.groupby('time')['sales'].sum()
total_sales_per_year

time
1994    40568
1995    46093
1996    42749
1997    51224
1998    54530
1999    56942
2000    55850
2001    58012
2002    63830
2003    64672
2004    69240
Name: sales, dtype: int64

In [6]:
df['total_sales_year'] = df['time'].map(total_sales_per_year)
df['market_share'] = df['sales'] / 70000  # or use df['total_sales_year'] if you want inside-only shares

In [7]:
# Inside share per year
inside_share = df.groupby('time')['market_share'].sum()
df['outside_share'] = df['time'].map(1 - inside_share)

In [8]:
df

,brand,model,sales,home,fuel,time,products,age,enginesize,income,averagesizecompetitor,price,total_sales_year,market_share,outside_share
0,BMW,5,160,0,0,1994,43,2,3,4045.563721,1870.260010,13251.009766,40568,0.002286,0.420457
1,BMW,3,210,1,1,1994,8,2,3,4045.563721,1782.649658,8996.839844,40568,0.003000,0.420457
2,daihatsu,applause,370,0,0,1994,34,4,2,4045.563721,1470.613037,5924.431152,40568,0.005286,0.420457
3,daihatsu,domino,120,0,1,1994,19,10,3,4045.563721,1907.504395,3472.578857,40568,0.001714,0.420457
4,daihatsu,charade,310,0,1,1994,22,5,2,4045.563721,1512.156616,4081.043701,40568,0.004429,0.420457
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1201,rover,mini,3980,1,0,2004,5,0,1,4989.745117,410.575714,6985.456055,69240,0.056857,0.010857
1202,rover,montego,1190,1,1,2004,17,3,3,4989.745117,924.141785,6929.085449,69240,0.017000,0.010857
1203,rover,800,130,1,0,2004,16,6,2,4989.745117,1084.296265,2936.483887,69240,0.001857,0.010857
1204,rover,200,110,0,0,2004,50,7,3,4989.745117,1464.663574,4492.992188,69240,0.001571,0.010857


## Step 2: Compute the mean utility for each product

Given the logit expression, we can compute the mean utility for each product through by taking the ratio of the natural logarithm between of $s_j$  to $s_0$ : 
$$\frac{s_j}{s_0} = \exp(\delta_j)$$  
$$\ln(s_j) - \ln(s_0) = \delta_j$$


In [9]:
df['log(market_share)'] = np.log(df['market_share'])
df['log(outside_share)'] = np.log(df['outside_share'])
df

,brand,model,sales,home,fuel,time,products,age,enginesize,income,averagesizecompetitor,price,total_sales_year,market_share,outside_share,log(market_share),log(outside_share)
0,BMW,5,160,0,0,1994,43,2,3,4045.563721,1870.260010,13251.009766,40568,0.002286,0.420457,-6.081077,-0.866413
1,BMW,3,210,1,1,1994,8,2,3,4045.563721,1782.649658,8996.839844,40568,0.003000,0.420457,-5.809143,-0.866413
2,daihatsu,applause,370,0,0,1994,34,4,2,4045.563721,1470.613037,5924.431152,40568,0.005286,0.420457,-5.242748,-0.866413
3,daihatsu,domino,120,0,1,1994,19,10,3,4045.563721,1907.504395,3472.578857,40568,0.001714,0.420457,-6.368759,-0.866413
4,daihatsu,charade,310,0,1,1994,22,5,2,4045.563721,1512.156616,4081.043701,40568,0.004429,0.420457,-5.419678,-0.866413
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1201,rover,mini,3980,1,0,2004,5,0,1,4989.745117,410.575714,6985.456055,69240,0.056857,0.010857,-2.867213,-4.522932
1202,rover,montego,1190,1,1,2004,17,3,3,4989.745117,924.141785,6929.085449,69240,0.017000,0.010857,-4.074542,-4.522932
1203,rover,800,130,1,0,2004,16,6,2,4989.745117,1084.296265,2936.483887,69240,0.001857,0.010857,-6.288716,-4.522932
1204,rover,200,110,0,0,2004,50,7,3,4989.745117,1464.663574,4492.992188,69240,0.001571,0.010857,-6.455770,-4.522932


In [10]:
df['mean_utility'] = df['log(market_share)'] - df['log(outside_share)'] 
df

,brand,model,sales,home,fuel,time,products,age,enginesize,income,averagesizecompetitor,price,total_sales_year,market_share,outside_share,log(market_share),log(outside_share),mean_utility
0,BMW,5,160,0,0,1994,43,2,3,4045.563721,1870.260010,13251.009766,40568,0.002286,0.420457,-6.081077,-0.866413,-5.214664
1,BMW,3,210,1,1,1994,8,2,3,4045.563721,1782.649658,8996.839844,40568,0.003000,0.420457,-5.809143,-0.866413,-4.942730
2,daihatsu,applause,370,0,0,1994,34,4,2,4045.563721,1470.613037,5924.431152,40568,0.005286,0.420457,-5.242748,-0.866413,-4.376335
3,daihatsu,domino,120,0,1,1994,19,10,3,4045.563721,1907.504395,3472.578857,40568,0.001714,0.420457,-6.368759,-0.866413,-5.502346
4,daihatsu,charade,310,0,1,1994,22,5,2,4045.563721,1512.156616,4081.043701,40568,0.004429,0.420457,-5.419678,-0.866413,-4.553265
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1201,rover,mini,3980,1,0,2004,5,0,1,4989.745117,410.575714,6985.456055,69240,0.056857,0.010857,-2.867213,-4.522932,1.655719
1202,rover,montego,1190,1,1,2004,17,3,3,4989.745117,924.141785,6929.085449,69240,0.017000,0.010857,-4.074542,-4.522932,0.448390
1203,rover,800,130,1,0,2004,16,6,2,4989.745117,1084.296265,2936.483887,69240,0.001857,0.010857,-6.288716,-4.522932,-1.765784
1204,rover,200,110,0,0,2004,50,7,3,4989.745117,1464.663574,4492.992188,69240,0.001571,0.010857,-6.455770,-4.522932,-1.932838


In [12]:
df.columns

Index(['brand', 'model', 'sales', 'home', 'fuel', 'time', 'products', 'age',
       'enginesize', 'income', 'averagesizecompetitor', 'price',
       'total_sales_year', 'market_share', 'outside_share',
       'log(market_share)', 'log(outside_share)', 'mean_utility'],
      dtype='object')

## Step 3: Estimate consumers’ preferences by regressing, the mean utility on cars characteristics

### Car Characteristics (Variable Mapping)

| Characteristic                          | Data Column                   |
|----------------------------------------|-------------------------------|
| Engine size                            | `enginesize`                  |
| Age                                     | `age`                         |
| Price                                   | `price`                       |
| Home-production dummy variable          | `home`                        |
| Gasoline dummy variable                 | `fuel`                        |
| Income                                  | `income`                      |
| Number of competitors’ products         | `products`                    |
| Number of scrapped cars                 | `time`                        |
| Average engine-size of competitors      | `averagesizecompetitor`       |

Questions:      
- Why is income a car characteristic? It's a market-level characteristic because it's the average income for that year. This is important to include to control for differences across years because consumers being richer in one year will affect what products they choose. 
- How to include brand and model? This combination is product j!  
- Given that price is correlated with unobserved characteristic which violates endogeneity, what is an instrument that is a good cost-shifter? My first thought is whether the car is imported or not that could easily be seen as correlated with unobserved quality. But, actually, referring back to (BLP, 1995), a good instrument is made of the product characteristics of competitors because **Competitor Characteristics → Competitor Costs → Competitor Prices → Your Price**  
- How to classify the scrapped cars variable? I don't it affects customer utility directly. In any case, I don't actually see it as a column so we can ignore for now. 

For regressions, we want to estimate are:

For OLS: 
$$\delta_j \equiv \ln\left(\frac{s_j}{s_0}\right) = x_j \beta - \alpha p_j + \xi_j$$

For IV:  

First Stage: 
$$p_j = z_j \pi + \eta_j$$  

Where: $z_j$ includes both $x_j$ (exogenous product characteristics)  
and instruments for $p_j$ (e.g., competitor characteristics) and $\eta_j$ is the first-stage error term.

Second Stage: 
$$\delta_j = x_j \beta - \alpha \hat{p_j} + \xi_j$$

Where: $\hat{p}_j$ is the predicted price from the first stage (i.e., the exogenous variation in price)  
and $\xi_j$ is the unobserved product quality (demand shock)

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

# Create a unique product identifier from brand and model
df['product_j'] = df['brand'] + '_' + df['model']

# Convert categorical variables if needed
for col in ['home', 'fuel']:
    if df[col].dtype == 'object':
        df[col] = pd.Categorical(df[col]).codes

# Check for NaN values and handle them
numeric_cols = ['enginesize', 'age', 'price', 'income', 'time', 'mean_utility']
for col in numeric_cols:
    if df[col].isna().any():
        print(f"Warning: NaN values detected in {col}")
        df[col] = df[col].fillna(df[col].mean())

# Formula approach - note the C() notation for categorical variables
formula = """
mean_utility ~ 1 + enginesize + age + price + home + fuel + income + C(time) + C(product_j)
"""

# Run OLS regression with formula
model = smf.ols(formula=formula, data=df)
results = model.fit()

# Display regression results
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:           mean_utility   R-squared:                       0.202
Model:                            OLS   Adj. R-squared:                  0.061
Method:                 Least Squares   F-statistic:                     1.432
Date:                Tue, 25 Mar 2025   Prob (F-statistic):           0.000473
Time:                        10:35:15   Log-Likelihood:                -2052.7
No. Observations:                1206   AIC:                             4469.
Df Residuals:                    1024   BIC:                             5397.
Df Model:                         181                                         
Covariance Type:            nonrobust                                         
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Inte

In [14]:
# Plot actual vs. predicted values
plt.figure(figsize=(10, 6))
plt.scatter(y, results.fittedvalues, alpha=0.5)
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'k--')
plt.xlabel('Actual Mean Utility')
plt.ylabel('Predicted Mean Utility')
plt.title('Actual vs. Predicted Mean Utility')
plt.tight_layout()
plt.show()

# Plot coefficient estimates for main variables (adjusted to match X variables)
main_vars = ['const', 'enginesize', 'age', 'price', 'home', 'fuel', 
             'income', 'time']
coefs = results.params[main_vars]
conf_ints = results.conf_int().loc[main_vars]
plt.figure(figsize=(12, 8))
plt.errorbar(range(len(coefs)), coefs, yerr=[coefs-conf_ints[0], conf_ints[1]-coefs], fmt='o')
plt.axhline(y=0, linestyle='--', color='k', alpha=0.7)
plt.xticks(range(len(coefs)), coefs.index, rotation=45)
plt.title('Regression Coefficients with 95% Confidence Intervals')
plt.tight_layout()
plt.show()

NameError: name 'y' is not defined

<Figure size 1000x600 with 0 Axes>

In [ ]:
# Note linearmodels automatically includes the exogenous regressors as controls in the first stage

# Add product identifier
df['product_j'] = df['brand'].astype(str) + '_' + df['model'].astype(str)

formula = """
mean_utility ~ 1 + enginesize + age + fuel + income + C(time) + C(product_j)
              + [price ~ averagesizecompetitor + products + home]
"""

# 3. Estimate via IV2SLS
iv2sls_model = IV2SLS.from_formula(formula, data=df)
results = iv2sls_model.fit()

# 4. View summary
print(results.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:           mean_utility   R-squared:                      0.6523
Estimator:                    IV-2SLS   Adj. R-squared:                 0.5872
No. Observations:                1206   F-statistic:                -1.903e+17
Date:                Sun, Mar 23 2025   P-value (F-stat)                1.0000
Time:                        14:08:52   Distribution:                chi2(190)
Cov. Estimator:                robust                                         
                                                                              
                                         Parameter Estimates                                          
                                    Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------------------------------
Intercept                             -4.2597     0.8044   

In [ ]:
exog_cols = exog.columns.tolist()
excluded_instrument_cols = ['averagesizecompetitor', 'products', 'home']

# Print any overlap
overlap = set(exog_cols).intersection(excluded_instrument_cols)
if overlap:
    print("Columns overlapping between exog and excluded instruments:", overlap)

# You can also check the final DataFrame for duplicates:
temp_instr = pd.concat([df_fe[excluded_instrument_cols], exog], axis=1)
duplicate_flags = temp_instr.columns.duplicated(keep=False)
if duplicate_flags.any():
    print("Duplicate columns in final instrument matrix:")
    print(temp_instr.columns[duplicate_flags])

In [ ]:
print(results.first_stage)

## Comment on the results of OLS vs IV

I started by including just averagesizecompetitor and products as instruments, but F < 10 so weak instruments were a concern. The partial R-squared was 0.028, meaning  that the instruments explain only ~2.8% of the variation in price conditional on other exogenous regressors.

However, after including the home-production dummy variable as another instrument as a cost-shifter, the F-Stat is now strong. 

Why home helps

Including home (domestic vs. foreign) likely improves identification because:
	•	It may proxy cost structure differences (e.g., labor, import taxes, shipping).
	•	Foreign-produced cars may be priced differently even for the same quality level.
	•	It’s likely exogenous to unobserved product quality, especially if it’s a supply-side characteristic.

This makes it a good cost shifter and valid instrument, assuming it’s not correlated with \xi_j (unobserved demand shocks). Is this a valid assumption?? 

## Notes on estimates
- They don't make sense as an increase in the age in 1 is associated in an increase in mean utility which is wrong!


### Estimate a nested logit model

The standard multinomial logit assumes IIA (Independence of Irrelevant Alternatives), which leads to unrealistic substitution patterns. The nested logit model relaxes this by allowing higher substitution within groups (nests) than across them. Therefore, the key idea is: Put products in the same nest when consumers are more likely to substitute between them.  


My chosen nests:
Nest 1: New vs Used Car  
I could do further nests based on brand, engine size, fuel type, domestically produced, but let's keep it simple!  